# Asyncio

## P1. `asyncio.new_event_loop()`

`asyncio` method to create a new event loop.

Yes, asyncio provides a method to create a task for a brand new event loop that hasn't started yet. This can be done using the `loop.create_task()` method, where `loop` is an instance of a new event loop. Here's how you can create a new event loop and schedule a task on it:

1. **Create a New Event Loop**: First, you create a new event loop using `asyncio.new_event_loop()`.
2. **Set the New Loop as the Current Loop**: You then set this new loop as the current event loop for the thread using `asyncio.set_event_loop(loop)`.
3. **Create a Task on the New Loop**: After setting the new loop, you can create a task on it using `loop.create_task(coroutine)`.
4. **Run the Loop**: Finally, you start the loop with `loop.run_until_complete(task)` or `loop.run_forever()`, depending on your needs.

Here's an example:

```python
import asyncio

# Define an asynchronous function (coroutine)
async def simulated_task():
    print("Task: Performing a task...")
    await asyncio.sleep(1)
    print("Task: Task completed.")

# Create a new event loop
new_loop = asyncio.new_event_loop()

# Set the new loop as the current loop
asyncio.set_event_loop(new_loop)

# Create a task on the new loop
task = new_loop.create_task(simulated_task())

# Run the loop until the task is completed
new_loop.run_until_complete(task)

# Close the loop
new_loop.close()
```

In this example, a brand new event loop is created and used to run the `simulated_task` coroutine. This is a common pattern when you need to run asynchronous code in environments where an event loop is not already running, such as in some scripts or non-async contexts.

## P1.1 Example with Callback and Await Task

I'll add a callback function and an `await task` line to the example, making it similar to the previous one. This will demonstrate how you can use callbacks with a newly created event loop.

```python
import asyncio

# Define an asynchronous function (coroutine)
async def simulated_task():
    print("Task: Performing a task...")
    await asyncio.sleep(1)
    print("Task: Task completed.")

# Callback function to be called when the simulated task is done
def on_task_complete(future):
    print("Callback: The task has completed!")

# Create a new event loop
new_loop = asyncio.new_event_loop()

# Set the new loop as the current loop
asyncio.set_event_loop(new_loop)

# Create a task on the new loop
task = new_loop.create_task(simulated_task())

# Attach the callback to the task
task.add_done_callback(on_task_complete)

# Run the loop until the task is completed
new_loop.run_until_complete(task)

# Close the loop
new_loop.close()
```

In this example, after creating the task, we attach the `on_task_complete` callback to it. The event loop then runs until the task is complete, at which point the callback is executed.

## P3. Differences Between `asyncio.new_event_loop()` and `asyncio.run(main())`

1. **Usage Context**:
   - `asyncio.new_event_loop()`: This approach is typically used in environments where you need more control over the event loop lifecycle or when you're integrating with existing applications that may have their own event loop management. It allows for manual setup and teardown of the event loop.
   - `asyncio.run(main())`: This is a high-level API for running a coroutine and managing the event loop. It's a simpler and more straightforward way to run async code, ideal for scripts or standalone async applications. It automatically creates a new event loop, runs the given coroutine, and closes the loop at the end.

2. **Event Loop Management**:
   - With `asyncio.new_event_loop()`, you have to manually create the loop, set it as the current loop, and ensure it's properly closed. This gives you more flexibility but also requires more boilerplate code.
   - `asyncio.run(main())` handles all these steps for you. It creates the loop, runs the coroutine, and closes the loop, all in one call. This makes the code cleaner and reduces the risk of errors like forgetting to close the loop.


3. **Flexibility and Control**:
   - Using `asyncio.new_event_loop()` gives you more control over when and how the event loop is run and closed. This can be important in complex applications where you need to integrate with other event-driven code.
   - `asyncio.run(main())` is more opinionated and straightforward, providing a standard way to run async code without needing to manage the event loop details.

In a non-Jupyter environment, if you're writing a simple script or a standalone async application, `asyncio.run(main())` is usually the preferred and easier method. However, if you need more control over the event loop or are integrating with other frameworks, manually managing the event loop with `asyncio.new_event_loop()` might be necessary.

## P4. Mock example related to bioinformatics

Certainly! Let's consider a bioinformatics scenario where you might have multiple long-running, I/O-bound tasks, such as querying different genomic databases or running computations on large datasets. Asynchronous programming can be particularly beneficial in this context because it allows you to execute these tasks concurrently, without blocking the rest of your program.

### Example Scenario: Asynchronous Genomic Data Fetching

**Objective**: Fetch data from multiple genomic databases concurrently and process the results once they're all retrieved.

**Tasks**:
1. Fetch variant information from a database like ClinVar.
2. Fetch gene expression data from a database like Gene Expression Omnibus (GEO).
3. Process the fetched data once both tasks are complete.

**Asynchronous Programming Advantage**:
- Both fetching tasks can run concurrently, significantly reducing the total time if they were run sequentially.
- The main program can continue to perform other tasks or remain responsive while waiting for the data.

### Example Code:

In [ ]:
# DO NOT RUN IN JUPYTER NOTEBOOK
# RUN IN A SCRIPT
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Run the event loop
asyncio.run(main())
# DO NOT RUN IN JUPYTER NOTEBOOK
# RUN IN A SCRIPT

### Explanation:

- **Asynchronous Functions**: `fetch_variant_data` and `fetch_gene_expression` simulate the asynchronous fetching of data. They use `await asyncio.sleep()` to mimic network delay.
- **Callbacks**: The `process_data` function is a callback that processes the data once it's fetched. It's attached to each task and is called once the task completes.
- **Concurrency**: The `asyncio.gather()` function waits for both tasks to complete, allowing them to run concurrently.

### Real-Life Application:

In a real-world scenario, you would replace the simulated fetching functions with actual asynchronous HTTP requests to the respective databases. 

The callback function could be more complex, involving data parsing, analysis, or integration with other data sources.

This approach is especially useful in bioinformatics when dealing with large-scale data analysis and multiple external data sources. Asynchronous programming allows for more efficient use of time and resources, particularly when dealing with I/O-bound tasks like network requests.

In [10]:
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Instead of using asyncio.run, use the existing event loop
loop = asyncio.get_event_loop()

try:
    print("try block")
    loop.run_until_complete(main()) # cannot be called on a loop that's already running #=> go to the except block
    # loop.create_task(main())
    print("end of try block")
except RuntimeError:
    print("Except block")
    # If the loop is closed, you need to reopen it
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.create_task(main())
    # Instead of run_forever, you may use loop.run_until_complete(main())
    print("End of except block")

try block
Except block
End of except block


/tmp/ipykernel_25853/1766756248.py:53: RuntimeWarning: coroutine 'main' was never awaited
  print("End of except block")


The warning you're seeing is due to a mismatch in how the event loop is being used in your script, specifically in the exception handling block. 

Here's a breakdown of the issue and how to resolve it:

1. **Issue:** 
   - The warning `RuntimeWarning: coroutine 'main' was never awaited` indicates that the coroutine `main()` was scheduled to run as a task (`loop.create_task(main())`), but it was never actually awaited or run to completion. 
   - This happens because after creating the task, you didn't start the loop with `loop.run_until_complete(main())` or `loop.run_forever()`.

2. **Solution:** 
   - In the `except` block, where you handle the `RuntimeError`, you should ensure that the event loop runs the `main()` coroutine to completion.
   - This can be achieved by replacing `loop.create_task(main())` with `loop.run_until_complete(main())`.

Here's the corrected portion of the script:

```python
try:
    loop.run_until_complete(main())
except RuntimeError:
    # If the loop is closed, you need to reopen it
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main())  # Run the main coroutine to completion
```

This ensures that whether you are in the normal execution flow or handling a `RuntimeError` due to a closed loop, the `main()` coroutine is always run to completion.

Is this because, in the except block, a new event loop was created and `loop.create_task(main())` is not enough to ensure that the event loop runs the `main() `coroutine to completion and that's why it needs to be replaced by `loop.run_until_complete(main())`?

Yes, your understanding is correct. Let's break it down for clarity:

1. **In the `except` Block**:
   - When a new event loop is created using `asyncio.new_event_loop()`, **it is not running by default**. This is a fresh event loop, independent of any that might be running in the background (like the one in a Jupyter Notebook).
   - Therefore, after creating and setting this new event loop, **you need to explicitly start it to run your coroutine**. This is where `loop.run_until_complete(main())` comes into play. It starts the new event loop and runs the `main()` coroutine to completion.
   - If you were to use `loop.create_task(main())` in this context without starting the loop (i.e., without `loop.run_until_complete(main())` or `loop.run_forever()`), the coroutine would be scheduled but would not execute because the loop isn't running.

In [19]:
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Instead of using asyncio.run, use the existing event loop
loop = asyncio.get_event_loop()

try:
    print("try block")
    loop.run_until_complete(main()) # cannot be called on a loop that's already running
    # loop.create_task(main())
    print("End of try block")
except RuntimeError:
    # If the loop from the jn was closed for some reason (never going to happend in a jn)
    print("Except block")
    # If the loop is closed, you need to reopen it
    loop = asyncio.new_event_loop()
    # set it as the current loop
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main()) # Run the main coroutine to completion
    # Instead of run_forever, you may use loop.run_until_complete(main())
    print("End of except block")

try block
Except block


RuntimeError: Cannot run the event loop while another loop is running

**THIS ALSO GAVE ME AN ERROR!**

The error message you're encountering is due to trying to run an event loop that is already running in the Jupyter Notebook environment. 

This is a common issue when working with asyncio in Jupyter Notebooks, as the notebook itself operates within an event loop.

Here’s a breakdown of what’s happening:

1. **In the `try` Block**: 
    - `loop.run_until_complete(main())` is called. 
        - However, in Jupyter Notebooks, there's typically an event loop already running in the background, which leads to the `RuntimeError: This event loop is already running`. 

2. **In the `except` Block**:
    - You correctly create a new event loop and set it as the current loop. Then, you try to run `main()` to completion using `loop.run_until_complete(main())`. 
        - However, because the Jupyter Notebook's event loop is still running, this leads to another `RuntimeError: Cannot run the event loop while another loop is running`.

To resolve this in a Jupyter Notebook environment, **you should schedule the coroutine as a task within the existing event loop**. 

i.e. **FIX THE TRY BLOCK** using `loop.create_task(main())` instead of `loop.run_until_complete(main())`

Here's how you can modify the script:

In [20]:
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Instead of using asyncio.run, use the existing event loop
loop = asyncio.get_event_loop()

# FIXING THE TRY BLOCK
try:
    print("try block")
    # loop.run_until_complete(main()) # cannot be called on a loop that's already running
    
    # Schedule main() as a task within the existing event loop
    loop.create_task(main())
    print("end of try block")
except RuntimeError:
    # This block is unlikely to ever be run in a Jupyter notebook environment because its loop is never closed, but ...
    print("Except block")
    # If the loop is closed (unlikely in Jupyter), you need to reopen it and run main
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop) 
    loop.run_until_complete(main()) #=> FAILS because the Jupyter Notebook's event loop is still running
    print("End of except block")

try block
end of try block


Fetching variant data...
Fetching gene expression data...
Variant data fetched!
Processing Variant Data...
Gene expression data fetched!
Processing Gene Expression Data...


The **try** block was fixed.

In this modification, the `try` block simply schedules `main()` as a task within the existing event loop, which is the correct approach in a Jupyter environment. 

If, for some reason, the loop is closed and you enter the except block, then you create a new loop and run `main()` to completion in this new loop.

This approach should work without raising the `RuntimeError`.

### **REVIEW 1**: To have it clear, how to schedule a task in an already running event loop

 (which is normal in Jupyter environments)

The error message you're encountering indicates a common issue when working with event loops in a Jupyter Notebook environment. Let's break down the error and the changes you made to the script.

1. **In the `try` Block**:
   - You have `loop.run_until_complete(main())`, but in a Jupyter Notebook, the event loop is typically already running. This is why you get the `RuntimeError: This event loop is already running`. In such an environment, `loop.run_until_complete(main())` is not appropriate because it tries to start an already running loop.

2. **In the `except` Block**:
   - You correctly create a new event loop and set it as the current loop. However, the issue arises with these two lines:
     - `loop.create_task(main())`
     - `loop.run_until_complete(main())`
   - The problem is that you're essentially scheduling the `main()` coroutine twice: first with `create_task`, and then again with `run_until_complete`. This is unnecessary and leads to the warning about a coroutine that was never awaited.

3. **Corrected Script**:
   - To correct the script, you should schedule the `main()` coroutine only once in the `except` block. Either use `create_task` and then start the loop with something like `loop.run_forever()`, or use `run_until_complete` directly.
   - Here is a revised version of the script:

     ```python
     # ... [rest of your code] ...

     # Instead of using asyncio.run, use the existing event loop
     loop = asyncio.get_event_loop()

     try:
         print("try block")
         # In a Jupyter environment, this line should be commented out or removed
         # loop.run_until_complete(main()) # cannot be called on a loop that's already running
         loop.create_task(main())
         print("end of try block")
     except RuntimeError:
         print("Except block")
         # If the loop is closed, you need to reopen it
         loop = asyncio.new_event_loop()
         asyncio.set_event_loop(loop)
         loop.run_until_complete(main())  # Correct way to run the coroutine on the new loop
         print("End of except block")
     ```

4. **Summary**:
   - In a Jupyter Notebook, use `loop.create_task(main())` without `run_until_complete` in the `try` block.
   - In the `except` block, where you're dealing with a fresh event loop, use `loop.run_until_complete(main())` to run the coroutine to completion.

This approach should avoid the errors and warnings you've encountered and allow your asyncio code to run smoothly in a Jupyter Notebook environment.

### **REVIEW 2**: When I change `loop.run_until_complete(main())` by `loop.create_task(main())`

... the `try` block is executed and everything runs well

You've made a very important observation. In a Jupyter Notebook environment, there is indeed already an event loop running, and this context makes the behavior of asyncio slightly different than in a standard Python script. 

1. **New Event Loop**: When you create a new event loop using `asyncio.new_event_loop()`, it is indeed a different loop from the one that Jupyter Notebook is already running. However, in the Jupyter environment, you typically don't need to create a new loop unless the default loop is closed or not suitable for some reason.

2. **Using `loop.create_task(main())`**: In the Jupyter Notebook, since an event loop is already running, using `loop.create_task(main())` schedules the `main()` coroutine on the existing event loop. This approach works **because the running event loop will pick up and execute the task you've scheduled**. This is why your `try` block works correctly and all the output is generated as expected.

    - **Run to Completion**: When you schedule a task on an existing event loop, as you did with `loop.create_task(main())`, the event loop continues to run and will execute this task. You don't need to manually start the loop again because it's already running.

3. **Behavior in Jupyter vs. Standard Python Script**: The key difference in a Jupyter Notebook environment is that you don't typically start and stop the event loop manually, as the notebook's underlying infrastructure is already handling an event loop. This contrasts with a standalone Python script, where you have more control over when and how the event loop starts and stops.

In summary, your understanding and adaptation of the asyncio code to work within the Jupyter Notebook environment is correct. 

The `try` block effectively **schedules** the `main()` coroutine **on the existing event loop**, which is **already running**, thereby avoiding the need to start a new loop or handle a `RuntimeError`.

### **Review 3**

What would happen if an event loop is **already running** (the one from the jupyter notebook environment?

Would `loop.create_task(main())` be enough to ensure that the event loop runs the `main()` coroutine to completion?

2. **In a Jupyter Notebook Environment**:
   - In the case where an event loop is already running (as in a Jupyter Notebook), `loop.create_task(main())` is generally sufficient to schedule and run the coroutine to completion.
   - The existing running event loop will pick up and execute the task you’ve scheduled. You don’t need to manually start the loop with `loop.run_until_complete(main())` or `loop.run_forever()` because the loop is already active and handling tasks.

3. **Key Differences**:
   - In a typical Python script, you have full control over the event loop: creating it, starting it, running tasks, and stopping it. 
      - You often use `asyncio.run(main())` or control the loop with `loop.run_until_complete(main())`.
   - In a Jupyter Notebook, the event loop is usually already running in the background. 
      - Therefore, you primarily focus on scheduling tasks on this existing loop, without needing to start or stop it.

So, to answer your final question: In a Jupyter Notebook, where an event loop is already running, `loop.create_task(main())` is **indeed enough to ensure that the event loop runs** the `main()` coroutine **to completion**. 

**The existing loop will automatically handle the execution of the task you schedule**.

### Experiment



If I change the last part of the script and ...

Add `loop.run_until_complete(main())` after `loop.create_task(main())` gives me an error message.

In [ ]:
# Instead of using asyncio.run, use the existing event loop
loop = asyncio.get_event_loop()

try:
    print("try block")
    loop.run_until_complete(main()) # cannot be called on a loop that's already running
    # loop.create_task(main())
    print("end of try block")
except RuntimeError:
    print("Except block")
    # If the loop is closed, you need to reopen it
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.create_task(main())
    loop.run_until_complete(main())
    # Instead of run_forever, you may use loop.run_until_complete(main())
    print("End of except block")

**Answer**

The error message you're encountering indicates a common issue when working with event loops in a Jupyter Notebook environment. Let's break down the error and the changes you made to the script.

1. **In the `try` Block**:
   - You have `loop.run_until_complete(main())`, but in a Jupyter Notebook, the event loop is typically already running. This is why you get the `RuntimeError: This event loop is already running`. In such an environment, `loop.run_until_complete(main())` is not appropriate because it tries to start an already running loop.

2. **In the `except` Block**:
   - You correctly create a new event loop and set it as the current loop. However, the issue arises with these two lines:
     - `loop.create_task(main())`
     - `loop.run_until_complete(main())`
   - The problem is that you're essentially scheduling the `main()` coroutine twice: first with `create_task`, and then again with `run_until_complete`. This is unnecessary and leads to the warning about a coroutine that was never awaited.

3. **Corrected Script**:
   - To correct the script, you should schedule the `main()` coroutine only once in the `except` block. Either use `create_task` and then start the loop with something like `loop.run_forever()`, or use `run_until_complete` directly.
   - Here is a revised version of the script:

     ```python
     # ... [rest of your code] ...

     # Instead of using asyncio.run, use the existing event loop
     loop = asyncio.get_event_loop()

     try:
         print("try block")
         # In a Jupyter environment, this line should be commented out or removed
         # loop.run_until_complete(main()) # cannot be called on a loop that's already running
         loop.create_task(main())
         print("end of try block")
     except RuntimeError:
         print("Except block")
         # If the loop is closed, you need to reopen it
         loop = asyncio.new_event_loop()
         asyncio.set_event_loop(loop)
         loop.run_until_complete(main())  # Correct way to run the coroutine on the new loop
         print("End of except block")
     ```

4. **Summary**:
   - In a Jupyter Notebook, use `loop.create_task(main())` without `run_until_complete` in the `try` block.
   - In the `except` block, where you're dealing with a fresh event loop, use `loop.run_until_complete(main())` to run the coroutine to completion.

This approach should avoid the errors and warnings you've encountered and allow your asyncio code to run smoothly in a Jupyter Notebook environment.

### **If the try block was not fixed** and if the except block has the line `loop.run_until_complete(main())`

"... because the Jupyter Notebook's event loop is still running, this leads to another `RuntimeError`: Cannot run the event loop while another loop is running."

**How can I correct the except block to avoid this?**

In [21]:
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Instead of using asyncio.run, use the existing event loop
loop = asyncio.get_event_loop()

# NOT FIXING THE TRY BLOCK
try:
    print("try block")
    loop.run_until_complete(main()) # cannot be called on a loop that's already running
    
    # Schedule main() as a task within the existing event loop
    # loop.create_task(main())
    print("end of try block")
except RuntimeError:
    print("Except block")
    # If the loop is closed (unlikely in Jupyter), you need to reopen it and run main
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    # loop.run_until_complete(main()) # FAILS because the Jupyter Notebook's event loop is still running
    loop.create_task(main()) # Also fails because the event loop was not started: 'coroutine 'main' was never awaited'
    print("End of except block")

try block
Except block
End of except block


/tmp/ipykernel_25853/2174216866.py:54: RuntimeWarning: coroutine 'main' was never awaited
  print("End of except block")


The warning you're encountering indicates that the coroutine `main()` was scheduled as a task in the event loop but was not awaited. This can happen when you use `loop.create_task(main())` but do not start the event loop with `loop.run_until_complete(main())` or `loop.run_forever()`. 

However, in a Jupyter Notebook environment, you typically don't need to start the event loop manually because it's already running.

Here's how you can modify your script to ensure that the coroutine `main()` is properly scheduled and executed within the Jupyter Notebook's event loop:

**ANSWER**: FIXING THE TRY BLOCK

In [22]:
import asyncio
import random

# Simulate fetching variant information
async def fetch_variant_data():
    print("Fetching variant data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Variant data fetched!")
    return "Variant Data"

# Simulate fetching gene expression data
async def fetch_gene_expression():
    print("Fetching gene expression data...")
    await asyncio.sleep(random.randint(1, 3))  # Simulate network delay
    print("Gene expression data fetched!")
    return "Gene Expression Data"

# Callback to process data after fetching
def process_data(future):
    data = future.result()
    print(f"Processing {data}...")

# Main function to run the event loop
async def main():
    # Create tasks for fetching data
    variant_task = asyncio.create_task(fetch_variant_data())
    gene_expression_task = asyncio.create_task(fetch_gene_expression())

    # Attach callbacks to each task
    variant_task.add_done_callback(process_data)
    gene_expression_task.add_done_callback(process_data)

    # Wait for all tasks to complete
    await asyncio.gather(variant_task, gene_expression_task)

# Use the existing event loop
loop = asyncio.get_event_loop()

# FIXING THE TRY BLOCK
try:
    print("try block")
    # Schedule main() as a task within the existing event loop
    loop.create_task(main())
    print("end of try block")
except RuntimeError:
    print("Except block")
    # In Jupyter, this block is unlikely to be executed EVER, but it's here as a fallback
    # Create a new event loop and set it as the current loop
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    # Schedule main() as a task within the new event loop
    loop.create_task(main())
    print("End of except block")

# No need to explicitly run the event loop in Jupyter Notebook

try block
end of try block


Fetching variant data...
Fetching gene expression data...
Variant data fetched!
Processing Variant Data...
Gene expression data fetched!
Processing Gene Expression Data...


In this modification:
- In the `try` block, we schedule `main()` as a task within the existing event loop. **This is typically the only block that will be executed in a Jupyter environment**.
- The `except` block is a **fallback** for environments where **the event loop is not already running**. However, **in a Jupyter Notebook, this block is unlikely to be needed**.
- There's no need to explicitly run the event loop with `run_until_complete` or `run_forever` since the **Jupyter Notebook environment should handle that for you**.

This approach should work in a Jupyter Notebook environment, allowing your asynchronous code to execute without raising the warning about the coroutine not being awaited.